In [1]:
%autosave 0

Autosave disabled


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('train_final.csv')
df.head()

,customer_ID,P_2_min,P_2_max,P_2_median,P_2_std,P_2_last,P_2_change,D_39_min,D_39_max,D_39_median,...,D_136_nulls,D_137_nulls,D_138_nulls,D_139_nulls,D_140_nulls,D_141_nulls,D_142_nulls,D_143_nulls,D_144_nulls,D_145_nulls
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.868580,0.960384,0.938469,0.024194,0.934745,-0.003724,0.001082,0.091505,0.002483,...,0.446319,0.446319,0.446319,-0.283183,-0.219693,-0.283183,0.596403,-0.283183,-0.219331,-0.283183
1,0005ef1480ffe4f0e5908eae07db8e05f2e705bca6a57d...,0.068484,0.186780,0.127632,0.083648,0.068484,-0.118296,0.031711,0.914384,0.473048,...,-3.166442,-3.166442,-3.166442,-0.283183,-0.219693,-0.283183,-1.582977,-0.283183,-0.219331,-0.283183
2,0005f2ab0cc352cd64eac9e60b981ed8c3289d20fa5fce...,0.837273,0.946487,0.894418,0.030962,0.875766,-0.018652,0.004118,0.740627,0.413056,...,0.446319,0.446319,0.446319,-0.283183,-0.219693,-0.283183,0.596403,-0.283183,-0.219331,-0.283183
3,0007ee1ac8edb6be588c6c8a158d2268e3228c49cf5277...,0.365222,0.646557,0.417593,0.081273,0.410590,-0.235967,0.000303,0.213325,0.006578,...,0.446319,0.446319,0.446319,-0.283183,-0.219693,-0.283183,0.596403,-0.283183,-0.219331,-0.283183
4,0008ef32824d3067b4d8ab70e7c79fa04c808f4d99194f...,0.752942,0.867515,0.843999,0.030110,0.850308,0.097366,0.000355,0.009881,0.007834,...,0.446319,0.446319,0.446319,1.033397,-0.219693,1.033397,0.596403,1.033397,-0.219331,1.033397


In [4]:
list(df.columns[df.isna().sum() > 0])

['P_2_std',
 'D_39_std',
 'B_1_std',
 'B_2_std',
 'R_1_std',
 'S_3_std',
 'D_41_std',
 'B_3_std',
 'D_42_std',
 'D_43_std',
 'D_44_std',
 'B_4_std',
 'D_45_std',
 'B_5_std',
 'R_2_std',
 'D_46_std',
 'D_47_std',
 'D_48_std',
 'D_49_std',
 'B_6_std',
 'B_7_std',
 'B_8_std',
 'D_50_std',
 'D_51_std',
 'B_9_std',
 'R_3_std',
 'D_52_std',
 'P_3_std',
 'B_10_std',
 'D_53_std',
 'S_5_std',
 'B_11_std',
 'S_6_std',
 'D_54_std',
 'R_4_std',
 'S_7_std',
 'B_12_std',
 'S_8_std',
 'D_55_std',
 'D_56_std',
 'B_13_std',
 'R_5_std',
 'D_58_std',
 'S_9_std',
 'B_14_std',
 'D_59_std',
 'D_60_std',
 'D_61_std',
 'B_15_std',
 'S_11_std',
 'D_62_std',
 'D_65_std',
 'B_16_std',
 'B_17_std',
 'B_18_std',
 'B_19_std',
 'B_20_std',
 'S_12_std',
 'R_6_std',
 'S_13_std',
 'B_21_std',
 'D_69_std',
 'B_22_std',
 'D_70_std',
 'D_71_std',
 'D_72_std',
 'S_15_std',
 'B_23_std',
 'D_73_std',
 'P_4_std',
 'D_74_std',
 'D_75_std',
 'D_76_std',
 'B_24_std',
 'R_7_std',
 'D_77_std',
 'B_25_std',
 'B_26_std',
 'D_78_std'

The standard deviation column is the main problem. The source of the problem was determined to be the default degrees of freedom when pandas calculates the standard deviation. Pandas assumes one delta degree of freedom, which divides the sum of squared deviations by n - 1 (n being the number of observation). The problem occurred with customers who had only 1 row of data, because the equation divided by 0 and produced a NaN value as a result. This could be addressed by switching to the population standard deviation, in which the sum of squared deviations is divided by n. This is not the correct fix: the customers in my dataset are samples and not the whole population. To handle this issue, the sample standard deviation will be calculated and some null values will be produced. The function responsible for the calculation will then impute 0 for all null values.

The null values in the categorical columns (or their inclusion at all) is due to unusual behavior from pd.get_dummies when run on multiple columns. Some of the categorical columns were unintentionally included in the dummy dataframe. I will correct this problem by adjusting the syntax used to call pd.get_dummies.

In [6]:
train = pd.read_csv('../../data/raw/train_data.csv', nrows=10)
train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


In [9]:
train.set_index('customer_ID', inplace=True)
train.drop(columns='S_2', inplace=True)

In [10]:
cat_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [12]:
pd.get_dummies(train, dummy_na=True)

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_140,D_141,D_142,D_143,D_144,D_145,D_63_CR,D_63_nan,D_64_O,D_64_nan
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,NaN,NaN,...,0.003706,0.003818,NaN,0.000569,0.000610,0.002674,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,NaN,NaN,...,0.003167,0.005032,NaN,0.009576,0.005492,0.009217,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,NaN,NaN,...,0.007329,0.000427,NaN,0.003429,0.006986,0.002603,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,NaN,NaN,...,0.004516,0.003200,NaN,0.008419,0.006527,0.009600,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,NaN,NaN,...,0.004946,0.008889,NaN,0.001670,0.008126,0.009827,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.945964,0.001746,0.007863,1.005006,0.004220,0.110946,0.009857,0.009866,NaN,NaN,...,0.008598,0.004529,NaN,0.000674,0.002223,0.002884,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.940705,0.002183,0.018859,1.008024,0.004509,0.103329,0.006603,0.000783,NaN,NaN,...,0.004361,0.009387,NaN,0.007727,0.007661,0.002225,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.914767,0.003029,0.014324,1.000242,0.000263,0.108115,0.009527,0.007836,NaN,NaN,...,0.008452,0.005553,NaN,0.001831,0.009616,0.007385,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.950845,0.009896,0.016888,1.003995,0.001789,0.102792,0.002519,0.009817,NaN,NaN,...,0.003968,0.007945,NaN,0.008722,0.004369,0.000995,1,0,1,0


When get_dummies is called on the entire dataframe, it only recognizes columns with string values. Many of my categorical columns are numerical and discrete, ie only contain whole numbers. I need to explicitly direct get_dummies' attention to all my categorical columns.

In [13]:
pd.get_dummies(train[cat_columns], dummy_na=True)

,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_66,D_68,D_63_CR,D_63_nan,D_64_O,D_64_nan
customer_ID,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,NaN,6.0,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,NaN,6.0,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,NaN,6.0,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,NaN,6.0,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,NaN,6.0,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,NaN,6.0,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,NaN,6.0,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,NaN,6.0,1,0,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,NaN,6.0,1,0,1,0


This doesn't improve my results, it simply returns the categorical and numeric columns back to me directly. I will specify them as the categorical datatype.

In [15]:
train[cat_columns].astype('category')

,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_66,D_68
customer_ID,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0


In [17]:
pd.get_dummies(train[cat_columns].astype('category'), dummy_na=True)

,B_30_0.0,B_30_nan,B_38_2.0,B_38_nan,D_114_1.0,D_114_nan,D_116_0.0,D_116_nan,D_117_4.0,D_117_nan,...,D_120_nan,D_126_1.0,D_126_nan,D_63_CR,D_63_nan,D_64_O,D_64_nan,D_66_nan,D_68_6.0,D_68_nan
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,1,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,1,0


By specifying my categorical columns should be given the type 'category', I'm able to break out my numerical categorical columns into dummies.